## This sector will transform the excel data into json file.
The json meta-data will contain:
"Company Name": 
  "Ticker": 
  "GICS Sector":
   "GICS Industry Group"
   


In [ ]:
import pandas as pd
import json

file_path = "data/GICS_list.xlsx"
excel_file = pd.ExcelFile(file_path)
excel_file.sheet_names
stock_data = pd.read_excel(file_path, sheet_name="Complete Stock List")
stock_data_filtered = stock_data[stock_data["Exchange"].isin(["New York Stock Exchange, Inc.", "Nasdaq"])].copy()
stock_data_filtered = stock_data_filtered[["Company Name", "Ticker", "GICS Sector", "GICS Industry Group"]]
stock_data_filtered = stock_data_filtered.copy()
stock_data_filtered.loc[:, "GICS Industry Group"] = stock_data_filtered["GICS Industry Group"].str.replace(" and ", " & ")
sorted_stock_data = stock_data_filtered.sort_values(by=["GICS Sector", "GICS Industry Group"])
stock_json_standardized = sorted_stock_data.to_dict(orient='records')

json_file_path = "data/stock_list.json"

with open(json_file_path, 'w') as json_file:
    json.dump(stock_json_standardized, json_file, indent=4)
    #print the json file path
    print(f"Json file created successfully at {json_file_path}")

## Show the GICS Sector and GICS Industry Group

![Fig](fig/GICS_map.png)

In [ ]:
# collect each of the unique GICS Sector and GICS Industry Group
unique_sector = sorted_stock_data["GICS Sector"].unique()
unique_industry = sorted_stock_data["GICS Industry Group"].unique()
# show as a list
unique_sector

In [ ]:
unique_industry

## Main crawler
We use the akshare stock data crawler to get the stock's history data. https://akshare.akfamily.xyz/data/stock/stock.html#id55
E.g.:
``` python
import akshare as ak
stock_us_hist_df = ak.stock_us_hist(symbol='106.TTE', period="daily", start_date="20200101", end_date="20240214", adjust="qfq")
print(stock_us_hist_df)
```

In [ ]:
import json
import os

input_json_path = 'data/stock_list.json'
with open(input_json_path, 'r') as file:
    stock_data = json.load(file)

# Group stocks by GICS Industry Group
industry_groups = {}
for entry in stock_data:
    industry = entry['GICS Industry Group']
    ticker = entry['Ticker']
    
    if industry not in industry_groups:
        industry_groups[industry] = []
    
    industry_groups[industry].append(entry)

# show the number of stock in each industry
# show as a markdown table:
print("| Industry | Number of Stocks |")
for industry, stocks in industry_groups.items():
    print(f"| {industry} | {len(stocks)} |")

#calculate the total number of stocks
total_stocks = sum([len(stocks) for stocks in industry_groups.values()])
print(f"Total number of stocks: {total_stocks}")


In [ ]:
import akshare as ak
us_stock_info_df = ak.stock_us_spot_em() 
valid_us_tickers = us_stock_info_df[['代码', '名称']]
input_json_path = 'data/stock_list.json'
with open(input_json_path, 'r') as file:
    stock_data = json.load(file)
us_stock_info_df['Cleaned Ticker'] = us_stock_info_df['代码'].str.replace(r'^\d+\.', '', regex=True)
code_mapping = dict(zip(us_stock_info_df['Cleaned Ticker'], us_stock_info_df['代码']))
success = 0
for entry in stock_data:
    current_ticker = entry['Ticker']
    # Check if the cleaned ticker exists in the mapping and update accordingly
    if current_ticker in code_mapping:
        entry['Ticker'] = code_mapping[current_ticker]
        print(f"Updated ticker for {entry['Company Name']}: {entry['Ticker']}")
        success += 1
    else:
        entry['Ticker'] = None
        print(f"Failed to update ticker for {entry['Company Name']}: {current_ticker}")
        
print(f"Updated tickers for {success} companies")
updated_json_path = "data/exist_stock_list.json"
filtered_stock_data = [entry for entry in stock_data if entry['Ticker'] is not None]

with open(updated_json_path, 'w') as json_file:
    json.dump(filtered_stock_data, json_file, indent=4)

In [ ]:

import re
# Dictionary to translate Chinese column names to English
column_translation = {
    "日期": "date",
    "开盘": "open",
    "收盘": "close",
    "最高": "high",
    "最低": "low",
    "成交量": "volume",
    "成交额": "turnover",
    "振幅": "amplitude",
    "涨跌幅": "price_change_percent",
    "涨跌额": "price_change_amount",
    "换手率": "turnover_rate"
}

def fetch_stock_data(symbol, output_file_path):
    try:
        stock_us_hist_df = ak.stock_us_hist(symbol=symbol, period="weekly", start_date="20170801", end_date="20240801", adjust="qfq")
        if stock_us_hist_df.empty:
            print(f"No data returned for symbol: {symbol}")
            return
        stock_us_hist_df.rename(columns=column_translation, inplace=True)
        stock_us_hist_df.to_json(output_file_path, orient='records', indent=4)
        print(f"Data saved for {symbol} at {output_file_path}")

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

base_directory = "data/GICS_Stock_Data"
if not os.path.exists(base_directory):
    os.makedirs(base_directory)
    
for entry in filtered_stock_data:
    industry = entry['GICS Industry Group']
    ticker = entry['Ticker']

    industry_directory = os.path.join(base_directory, industry)
    if not os.path.exists(industry_directory):
        os.makedirs(industry_directory)
    clean_name = f"{ticker}"
    clean_name = re.sub(r'^\d+\.', '', clean_name)
    stock_file_path = os.path.join(industry_directory, clean_name + ".json")

    # Fetch and save stock data
    fetch_stock_data(symbol=ticker, output_file_path=stock_file_path)